In [4]:
import numpy as np
import pandas as pd

import os
import sys

# Get the current working directory (where the notebook is located)
notebook_dir = os.getcwd()

# If you need to add a specific directory relative to the notebook directory, adjust the path accordingly
desired_path = os.path.join(notebook_dir, 'relative/path/to/directory')

# Add the directory to sys.path
sys.path.insert(0, desired_path)


### What we need to load/create the source Datasets with indicators and signals

In [ ]:

from jgtpy import JGTCDSSvc as cdssvc



### From bellow, we import what we will require to add MFI lags on the TTF etc

In [21]:

from jgtutils.jgtconstants import MFI_VAL,MFI_SIGNAL,VOLUME,FDB_TARGET as TARGET
from jgtpy import mfihelper as mfih
from jgtpy.mfihelper import get_mfi_features_column_list_by_timeframe
from jgtml import anhelper



* Create the dataset

In [5]:
i="SPX500";t="D1"


In [6]:
df1=cdssvc.get_higher_cdf(i,t)
#@STCIssue That is not what we NEED

In [20]:
#df1
# NOT WHAT WE WANT

### --@STCIssue might be from ptottf import read_ttf_csv


In [9]:

from jgtml.ptottf import read_ttf_csv
#@STCIssue THAT is What we need
ttfdf=read_ttf_csv(i, t, use_full=True)

   Read TTF:  /var/lib/jgt/full/data/ttf/SPX500_D1_ttf.csv


In [12]:
ttfdf.columns

Index(['BidOpen', 'BidHigh', 'BidLow', 'BidClose', 'AskOpen', 'AskHigh',
       'AskLow', 'AskClose', 'Volume', 'Open', 'High', 'Low', 'Close',
       'Median', 'ao', 'ac', 'jaw', 'teeth', 'lips', 'bjaw', 'bteeth', 'blips',
       'tjaw', 'tteeth', 'tlips', 'fh', 'fl', 'fh3', 'fl3', 'fh5', 'fl5',
       'fh8', 'fl8', 'fh13', 'fl13', 'fh21', 'fl21', 'fh34', 'fl34', 'fh55',
       'fl55', 'fh89', 'fl89', 'mfi', 'fdbb', 'fdbs', 'fdb', 'aoaz', 'aobz',
       'zlc', 'zlcb', 'zlcs', 'zcol', 'sz', 'bz', 'acs', 'acb', 'ss', 'sb',
       'price_peak_above', 'price_peak_bellow', 'ao_peak_above',
       'ao_peak_bellow', 'mfi_sq', 'mfi_green', 'mfi_fade', 'mfi_fake',
       'mfi_sig', 'mfi_str', 'mfi_str_M1', 'zcol_M1', 'ao_M1', 'mfi_str_W1',
       'zcol_W1', 'ao_W1'],
      dtype='object')

In [14]:
ttfdf.tail(3)

,BidOpen,BidHigh,BidLow,BidClose,AskOpen,AskHigh,AskLow,AskClose,Volume,Open,...,mfi_fade,mfi_fake,mfi_sig,mfi_str,mfi_str_M1,zcol_M1,ao_M1,mfi_str_W1,zcol_W1,ao_W1
Date,,,,,,,,,,,,,,,,,,,,,
2024-07-02 21:00:00,5513.96,5547.01,5511.19,5542.12,5514.36,5547.51,5511.79,5542.62,51015,5514.16,...,1,0,3,--,-+,gray,0.822879,-+,gray,0.803511
2024-07-03 21:00:00,5542.12,5548.70,5537.23,5540.25,5542.62,5549.30,5537.73,5540.75,12133,5542.37,...,0,1,2,-+,-+,gray,0.822879,-+,gray,0.803511
2024-07-04 21:00:00,5540.25,5577.70,5536.90,5568.90,5540.75,5578.21,5537.50,5569.50,72693,5540.50,...,0,0,1,+-,-+,gray,0.822879,-+,gray,0.803511


### Adding Lags to data we wanted

* Added lags
* ending up with probably one row that we pass to the predictor of the model we created with the MX

In [18]:
# Get columns list we want to add lags to
from jgtpy.mfihelper import get_mfi_features_column_list_by_timeframe

columns_to_add_lags_to = get_mfi_features_column_list_by_timeframe(t)
columns_to_add_lags_to

['mfi_str_M1', 'mfi_str_W1', 'mfi_str']

In [24]:

# Add lag from anhelper
ttfdf=anhelper.add_lagging_columns(ttfdf, columns_to_add_lags_to)

In [25]:
ttfdf.columns

Index(['BidOpen', 'BidHigh', 'BidLow', 'BidClose', 'AskOpen', 'AskHigh',
       'AskLow', 'AskClose', 'Volume', 'Open', 'High', 'Low', 'Close',
       'Median', 'ao', 'ac', 'jaw', 'teeth', 'lips', 'bjaw', 'bteeth', 'blips',
       'tjaw', 'tteeth', 'tlips', 'fh', 'fl', 'fh3', 'fl3', 'fh5', 'fl5',
       'fh8', 'fl8', 'fh13', 'fl13', 'fh21', 'fl21', 'fh34', 'fl34', 'fh55',
       'fl55', 'fh89', 'fl89', 'mfi', 'fdbb', 'fdbs', 'fdb', 'aoaz', 'aobz',
       'zlc', 'zlcb', 'zlcs', 'zcol', 'sz', 'bz', 'acs', 'acb', 'ss', 'sb',
       'price_peak_above', 'price_peak_bellow', 'ao_peak_above',
       'ao_peak_bellow', 'mfi_sq', 'mfi_green', 'mfi_fade', 'mfi_fake',
       'mfi_sig', 'mfi_str', 'mfi_str_M1', 'zcol_M1', 'ao_M1', 'mfi_str_W1',
       'zcol_W1', 'ao_W1', 'mfi_str_M1_lag_1', 'mfi_str_M1_lag_2',
       'mfi_str_M1_lag_3', 'mfi_str_M1_lag_4', 'mfi_str_M1_lag_5',
       'mfi_str_W1_lag_1', 'mfi_str_W1_lag_2', 'mfi_str_W1_lag_3',
       'mfi_str_W1_lag_4', 'mfi_str_W1_lag_5', 'mfi_str_l

In [26]:
ttfdf.tail(3)

,BidOpen,BidHigh,BidLow,BidClose,AskOpen,AskHigh,AskLow,AskClose,Volume,Open,...,mfi_str_W1_lag_1,mfi_str_W1_lag_2,mfi_str_W1_lag_3,mfi_str_W1_lag_4,mfi_str_W1_lag_5,mfi_str_lag_1,mfi_str_lag_2,mfi_str_lag_3,mfi_str_lag_4,mfi_str_lag_5
Date,,,,,,,,,,,,,,,,,,,,,
2024-07-02 21:00:00,5513.96,5547.01,5511.19,5542.12,5514.36,5547.51,5511.79,5542.62,51015,5514.16,...,-+,-+,--,--,--,-+,--,++,++,++
2024-07-03 21:00:00,5542.12,5548.70,5537.23,5540.25,5542.62,5549.30,5537.73,5540.75,12133,5542.37,...,-+,-+,-+,--,--,--,-+,--,++,++
2024-07-04 21:00:00,5540.25,5577.70,5536.90,5568.90,5540.75,5578.21,5537.50,5569.50,72693,5540.50,...,-+,-+,-+,-+,--,-+,--,-+,--,++


## _mfi_str_add_lag_as_int


In [27]:
#_mfi_str_add_lag_as_int
from jgtml.mxhelper import _mfi_str_add_lag_as_int


ttfdf=_mfi_str_add_lag_as_int(ttfdf,t)

ValueError: invalid literal for int() with base 10: '+-'

AttributeError: 'tuple' object has no attribute 'columns'